In [1]:
import numpy as np

In [3]:
pre1 = lambda free, bound, dicer, k1, k2 : bound*k2 - free*dicer*k1

In [4]:
def auto_generate_ODEs(n):# free, bound, reaction_rates, dicer):
    """
    Function to automatically generate ODEs for model system
    
    Args:
    n (int):                        number of mirnas
    #free (list of floats):          concentration of initial free miRNA
    #bound (list floats):            concentration of initial bound miRNA
    #reaction_rates(list of tuples): reaction rate for binding, unbinding, and 
                                    maturation of specific miRNA species
    #dicer (float):                  dicer concentration
    """
    
    frees = []
    bounds = []
    matures = []
    
    for i in range(0, n):
        frees.append(lambda free, bound, dicer, k1, k2 : bound*k2 - free*dicer*k1)
        bounds.append(lambda free, bound, dicer, k1, k2 : free*dicer*k1 - bound*k2)
        matures.append(lambda bound, k3 : bound*k3)
                       
    adds = []
    retracts = []
    
    for i in range(0, n):
        adds.append(lambda bound, k2, k3 : bound*(k2 + k3))
        retracts.append(lambda free, dicer, k1 : free*dicer*k1)
    
    return frees, bounds, matures, adds, retracts